In [2]:
%matplotlib inline
import datacube

In [3]:
dc = datacube.Datacube()

In [4]:
dc.list_products()

,name,description,product_type,platform,lat,format,time,instrument,lon,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,
1,ls5_usgs_ard,Landsat 5 USGS ARD 30 metre tile,LS_USGS_ARD,LANDSAT_5,None,GeoTiff,None,TM,None,"PROJCS[""Albers"", GEOGCS[""NAD83"", DATUM[""North_...","[-30, 30]",None,"(y, x)"


In [22]:
bands = dc.list_measurements()
requested = {'red', 'swir1'}
band_listing = bands[bands['aliases'].apply(lambda b: bool(requested.intersection(b)))]
band_listing

aliases  dtype flags_definition      name  \
product      measurement                                                      
ls5_usgs_ard sr_band3       [band_3, red]  int16              NaN  sr_band3   
             sr_band5     [band_5, swir1]  int16              NaN  sr_band5   

                          nodata  \
product      measurement           
ls5_usgs_ard sr_band3      -9999   
             sr_band5      -9999   

                                                        spectral_definition  \
product      measurement                                                      
ls5_usgs_ard sr_band3     {'response': [0.0018, 0.002, 0.0079, 0.0205, 0...   
             sr_band5     {'response': [0.0012, 0.0013, 0.0014, 0.0016, ...   

                         units  
product      measurement        
ls5_usgs_ard sr_band3        1  
             sr_band5        1

In [54]:
measurements = band_listing.index.get_level_values(1).tolist()
measurements

['sr_band3', 'sr_band5']

In [64]:
data = dc.load(
    product='ls5_usgs_ard', 
    measurements=measurements, 
    dask_chunks={'x':5000, 'y':1}, 
#     time=('1991-03-18', '1991-04-20'),
)

In [66]:
drill = data.isel(x=0, y=0)

In [68]:
drill = drill.sr_band3[:10]

In [69]:
drill.load()

<xarray.DataArray 'sr_band3' (time: 10)>
array([-9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999, -9999], dtype=int16)
Coordinates:
  * time     (time) datetime64[ns] 1986-08-05T17:59:06 1990-03-25T17:57:20 ...
    y        float64 3.015e+06
    x        float64 -1.816e+06
Attributes:
    nodata: -9999
    units: 1
    crs: PROJCS["Albers", GEOGCS["NAD83", DATUM["North_American_Datum_1983", SPHEROID["GRS 1980",6378137,298.2572221010042, AUTHORITY["EPSG","7019"]], AUTHORITY["EPSG","6269"]], PRIMEM["Greenwich",0], UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4269"]], PROJECTION["Albers_Conic_Equal_Area"], PARAMETER["standard_parallel_1",29.5], PARAMETER["standard_parallel_2",45.5], PARAMETER["latitude_of_center",23], PARAMETER["longitude_of_center",-96], PARAMETER["false_easting",0], PARAMETER["false_northi...
    spectral_definition: {'response': [0.0018, 0.002, 0.0079, 0.0205, 0.0375, 0.0409, 0.0555, 0.073, 0.0906, 0.1082, 0.1262, 0.1444, 0.1626, 0.2169, 0.2958, 0.3

In [56]:
datasets = dc.find_datasets(product='ls5_usgs_ard', time=('1991-03-18', '1991-04-20'))

In [51]:
dataset = datasets[0]
dataset.measurements['sr_band1']['path']

's3://test-odc-bucket/LT05_L1TP_044027_19910319_20160929_01_T1_sr_band1.tif'

In [60]:
data.isel(x=-1, y=-1)

<xarray.Dataset>
Dimensions:   (time: 11)
Coordinates:
  * time      (time) datetime64[ns] 1991-03-19T18:04:47 1991-03-26T18:10:39 ...
    y         float64 2.865e+06
    x         float64 -1.666e+06
Data variables:
    sr_band3  (time) int16 2849 -9999 -9999 -9999 -9999 -9999 -9999 953 ...
    sr_band5  (time) int16 2810 -9999 -9999 -9999 -9999 -9999 -9999 2677 ...
Attributes:
    crs: PROJCS["Albers", GEOGCS["NAD83", DATUM["North_American_Datum_1983", SPHEROID["GRS 1980",6378137,298.2572221010042, AUTHORITY["EPSG","7019"]], AUTHORITY["EPSG","6269"]], PRIMEM["Greenwich",0], UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4269"]], PROJECTION["Albers_Conic_Equal_Area"], PARAMETER["standard_parallel_1",29.5], PARAMETER["standard_parallel_2",45.5], PARAMETER["latitude_of_center",23], PARAMETER["longitude_of_center",-96], PARAMETER["false_easting",0], PARAMETER["false_northi...